In [1]:
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from  keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import copy

import sklearn.model_selection
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

/home/ubuntu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("./train_cleaned_no-stopwords.csv")
test = pd.read_csv("./test_cleaned_no-stopwords.csv")
list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [3]:
def count_words(corpus):
    word_counts = {}
    for sentence in corpus:
        for word in set(sentence.split()):
                if word not in word_counts: word_counts[word] = 0
                word_counts[word] += 1
    return word_counts

def remove_low_freq_words(text, val_w):
    text = set(text.split())
    text = list(text & val_w)
    if len(text) == 0:
        text = ["CVxTz"]
    text = " ".join(text)    
    return text
        
all_text = list(list_sentences_train) + list(list_sentences_test)
print len(all_text)
word_counts = count_words(all_text)
print len(word_counts)

val_words = set()
for k, v in word_counts.items():
    if v > 1:
        val_words.add(k)
print(len(val_words))
list_sentences_f_train  = ["CVxTz"] * len(list_sentences_train)
list_sentences_f_test   = ["CVxTz"] * len(list_sentences_test)

for i in tqdm(range(len(list_sentences_train))):
    list_sentences_f_train[i] = remove_low_freq_words(list_sentences_train[i], val_words)

for i in tqdm(range(len(list_sentences_test))):
    list_sentences_f_test[i] = remove_low_freq_words(list_sentences_test[i], val_words)

312735
203883


  3%|▎         | 5443/159571 [00:00<00:02, 54370.68it/s]

101897


100%|██████████| 153164/153164 [00:02<00:00, 56325.20it/s]


In [4]:
all_sentences = list_sentences_f_train + list_sentences_f_test

In [5]:
len(all_sentences)

312735

In [6]:
all_sentences[0]

'retired vandalisms since please edits voted template new reverted dolls username explanation gas fan york fac hardcore metallica closure made remove page talk'

In [7]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

def splitter(x): return x.split(" ")

all_sentences_bagofwords = vec.fit_transform(all_sentences)


In [8]:
this_lda = LDA()
this_lda.fit(all_sentences_bagofwords)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [9]:
train_bow = vec.transform(list_sentences_f_train)
test_bow = vec.transform(list_sentences_f_test)

train_lda = this_lda.transform(train_bow)
test_lda = this_lda.transform(test_bow)



In [10]:

pd.DataFrame(np.column_stack([train["id"].values, train_lda])).to_csv("/home/ubuntu/kaggle/toxicity/team_ensembl/Florian/out/train_1703_FG_LDA.csv", header= False, index= False)
pd.DataFrame(np.column_stack([test["id"].values, test_lda])).to_csv("/home/ubuntu/kaggle/toxicity/team_ensembl/Florian/out/test_1703_FG_LDA.csv", header= False, index= False)   

In [17]:
train_lda[0]


array([0.00416695, 0.0458333 , 0.05962139, 0.00416712, 0.55320212,
       0.0041667 , 0.00416667, 0.00416746, 0.00416752, 0.31634078])

In [18]:
train_lda[6]

array([0.4199744 , 0.02000002, 0.02000143, 0.02000212, 0.42      ,
       0.02      , 0.02000301, 0.0200013 , 0.02000495, 0.02001277])

In [16]:
y[6]

array([1, 1, 1, 0, 1, 0])